DataJoint Primer. Section 2.
# Defining a table

DataJoint relies the *Relational Data Model* for organizing its data.

The crux of the Relational Data Model is that all data are stored in simple **tables** with **rows** and **columns**.

In more theoretical and academic settings, he tables are called **relations**, columns are **attributes**, and rows are **tuples**.  We may use boths sets of terms interchangeably.

The tables are simple and may not be nested and can always be accessed directly without following paths of links.

Groups of related tables are called **schemas**. The word **database** is synonymous to **schema** in this context and we may use them interchangeably.

To summarize, when speaking about databases
> "schema" == "database"

> "relation" == "table"

> "attribute" == "column"

> "tuple" == "row"

As described in Primer 0, I have all privileges to any schema that starts with `dimitri_`.

Therefore, let me create a new schema for data about things in our lab. 

In [2]:
import datajoint as dj
schema = dj.schema('dimitri_lab', locals())

Now I will create a table in the database `dimitri_lab` by declaring a special Python class. For example, let's create the table `Person` to store information about people who work in our lab.

In [3]:
@schema
class Person(dj.Manual):
    definition = """   # members of the lab
    username : char(16)   # short unique name
    ----
    full_name : varchar(25)
    """

This defines a table with two columns `username` and `full_name`.  We can now enter single row of datas into `Person` one at a time using the `insert1` method:

In [13]:
Person().insert1(('dimitri', 'Dimitri Yatsenko'))
Person().insert1(dict(username='shan', full_name="Shan Shen"))

Or multiple rows at once using the `insert` method:

In [14]:
Person().insert((
    ('jake', 'Jacob Reimer'),
    ('fabee', 'Fabian Sinz'),
    {'full_name': 'Edgar Y. Walker', 'username': 'edgar'}
    ))

You may get a preview of the contents of the table from the instance of its class:

In [17]:
Person()

username,full_name
dimitri,Dimitri Yatsenko
edgar,Edgar Y. Walker
fabee,Fabian Sinz
jake,Jacob Reimer
shan,Shan Shen


DataJoint will not allow entering two rows with the same `username` because it is the *primary key* of `Person`.

The primary key attributes are listed above the separator `---` in the `definition` string.  Every row must have unique values in the primary key attributes.

In [18]:
Person().insert1(('jake', 'Jacob W'))

IntegrityError: (1062, "Duplicate entry 'jake' for key 'PRIMARY'")

# A closer look

## What makes a relation class
Note that:
- Class `Person` is decorated with the `schema` object created earlier. It links the table to the database. 
- Class `Person` inherits from `dj.Manual`, which indicates that data will be entered into `Person` manually.  We will discuss automatically populated tables in future sections.
- Class `Person` defines the multiline string property `definition`, which defines the attributes (columns) of the table.
- The class name **must** be in CamelCase and can only contain alphanumerical characters (underscores are not allowed).  This is important because datajoint converts the class name into the corresponding table name in the database.

## Format of the table definition string
Let's look at the `definition` string closely:

The first line beginning with a `#` describes the contents of the table. 
Each of the subsequent lines defines an attribute (column) of the table in the format
> `attribute_name : type    # comment`

All attribute names must be in lowercase, must start with a letter and can only contain alphanumerical characters and underscores.

The separator 
> `----`

comprises three or more dashes and separates the **primary key** attributes above from the **dependent attributes** below. 

The primary key attributes uniquely identify each row in the table; the table cannot contain two rows with the same values of the primary key attributes.

## Attribute types
The attribute types are MySQL data types and are summarized here http://datajoint.github.io/datatypes/. 

Most commonly used in datajoint are 
- signed integers: ** `tinyint`, `smallint`, `int`, `bigint`  **
- unsigned integers: ** `tinyint unsigned`, `smallint unsigned`, `int unsigned`, `bigint unsigned` ** 
- floating-point and fixed-point fractional numbers: **`float`, `double`, `decimal`**
- true/false **`boolean`**
- strings **`char`, `varchar`**
- dates and times **`date`, `timestamp`**
- arbitrary things such as images, traces, etc: **`longblob`** 